### Goal
We are going to build a model for predicting a price for apartments and condominiums in Singapore.

##### Task for the current workbook:
- Build some models for predicting the price, based on the data prepared in the workbook 1 and the insights into the data in the workbook 2

#### Strategy
- Based on the analysis in the Workbook 2, we concluded that linear models are not suitable for this dataset because of significant non-linearities in the data.
- Therefore, we will be using models which are more suitable for non-linear data, such as Decision Tree regression, Random forest regression, and artificial neural networks.
- Using some variables such as project_name, street and postal_code will lead to high-dimensional binary vector spaces. We can use two strategies to deal with this:
    - Try models without these variables. Latitude and longitude may partly [replace them]
    - Use dimensionality reduction, e.g. PCA, and run the regressions on that data
- Our data includes several years of observations. From the analysis we have seen that the prices have changed over the years. Therefore, we may test how robust the model is by choosing y1 as a train set and y2 as a test set

#### Plan:
- Use data cleaned from outliers
- Numeric and geographical data only:
    - Decision tree regression
    - Random forest regression
- All data:
    - PCA:
        - Generate principal components. Try different number of PC, make sure that explained_variance_ratio_ cummulative is sufficient (?)
        - Decision tree regression
        - Random forest regression
    - ANN:
        - Run ANN on all data

### Importing data

In [212]:
import pandas as pd
import numpy as np

data_path = "data/"
file_leases_no_outliers = "2_lease_no_outliers.csv" 
# file_freehold_no_outliers = "2_freehold_no_outliers.csv" 
TRAIN_SIZE = 0.7

In [213]:
df = pd.read_csv(data_path+file_leases_no_outliers, parse_dates=['sale_date', 'tenure_start'])

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68190 entries, 0 to 68189
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   project_name        68190 non-null  object        
 1   street              68190 non-null  object        
 2   postal_district     68190 non-null  int64         
 3   sale_type           68190 non-null  object        
 4   area_type           68190 non-null  object        
 5   property_type       68190 non-null  object        
 6   tenure_type         68190 non-null  object        
 7   mkt_segment         68190 non-null  object        
 8   sale_month          68190 non-null  int64         
 9   sale_year           68190 non-null  int64         
 10  sale_date           68190 non-null  datetime64[ns]
 11  tenure_start        68190 non-null  datetime64[ns]
 12  tenure_start_y_ago  68190 non-null  float64       
 13  tenure_years        68190 non-null  float64   

In [215]:
df.tenure_type.unique()

array(['Lease'], dtype=object)

##### Regression on variables excluding project_name and street

In [56]:
df_clean = df.drop(columns=['project_name', 'street', 'sale_date', 'tenure_start', 
                             'tenure_years', 'lat', 'lon', 'price_total' ])
dataset_1 = df_clean

In [57]:
dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68190 entries, 0 to 68189
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   postal_district     68190 non-null  int64  
 1   sale_type           68190 non-null  object 
 2   area_type           68190 non-null  object 
 3   property_type       68190 non-null  object 
 4   tenure_type         68190 non-null  object 
 5   mkt_segment         68190 non-null  object 
 6   sale_month          68190 non-null  int64  
 7   sale_year           68190 non-null  int64  
 8   tenure_start_y_ago  68190 non-null  float64
 9   tenure_remained     68190 non-null  float64
 10  floor_level         68190 non-null  float64
 11  max_floor           68190 non-null  float64
 12  area_sqft           68190 non-null  float64
 13  lat_adj             68190 non-null  float64
 14  lon_adj             68190 non-null  float64
 15  price_sqft          68190 non-null  float64
dtypes: f

In [35]:
dataset_1 = dataset_1.values

In [36]:
X = dataset_1[:, :-1]
y = dataset_1[:, -1]

##### Categorical columns

In [37]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [38]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3 , 4, 5, 6, 7])], 
                       remainder='passthrough')

In [39]:
X = ct.fit_transform(X).toarray()

### Entire dataset approach

##### Split training and test set

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = TRAIN_SIZE)

##### Feature scaling

In [74]:
from sklearn.preprocessing import StandardScaler

In [76]:
sc = StandardScaler()

In [77]:
X_train[:, -7:] = sc.fit_transform(X_train[:, -7:])

In [78]:
X_test[:, -7:] = sc.transform(X_test[:, -7:])

In [79]:
#save backup
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()
y_train_copy = y_train.copy()
y_test_copy = y_test.copy()

#### Regression models

In [80]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

##### Decision tree

In [90]:
tree_1 = DecisionTreeRegressor()
tree_1.fit(X_train, y_train)
y_pred_tree_1 = tree_1.predict(X_test)

In [91]:
r2_score(y_test, y_pred_tree_1)

0.9674631023066488

##### Random forest

In [92]:
forest_1 = RandomForestRegressor()
forest_1.fit(X_train, y_train)
y_pred_forest_1 = forest_1.predict(X_test)

In [93]:
r2_score(y_test, y_pred_forest_1)

0.9801877764273224

### Time-split approach

In [73]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68190 entries, 0 to 68189
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   postal_district     68190 non-null  int64  
 1   sale_type           68190 non-null  object 
 2   area_type           68190 non-null  object 
 3   property_type       68190 non-null  object 
 4   tenure_type         68190 non-null  object 
 5   mkt_segment         68190 non-null  object 
 6   sale_month          68190 non-null  int64  
 7   sale_year           68190 non-null  int64  
 8   tenure_start_y_ago  68190 non-null  float64
 9   tenure_remained     68190 non-null  float64
 10  floor_level         68190 non-null  float64
 11  max_floor           68190 non-null  float64
 12  area_sqft           68190 non-null  float64
 13  lat_adj             68190 non-null  float64
 14  lon_adj             68190 non-null  float64
 15  price_sqft          68190 non-null  float64
dtypes: f

In [85]:
dataset_2_train = df_clean[df_clean.sale_year == 2022].drop(columns=['sale_year']).values
dataset_2_test = df_clean[df_clean.sale_year == 2023].drop(columns=['sale_year']).values
X_train_2 = dataset_2_train[:, :-1]
y_train_2 = dataset_2_train[:, -1]
X_test_2 = dataset_2_test[:, :-1]
y_test_2 = dataset_2_test[:, -1]

#### Feature scaling and column transforming

In [86]:
# one column less, as sale_year column is removed
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3 , 4, 5, 6])], 
                       remainder='passthrough')
X_train_2 = ct.fit_transform(X_train_2).toarray()
X_test_2 = ct.transform(X_test_2).toarray()

In [ ]:
sc_2 = StandardScaler()
X_train_2[:, -7:] = sc_2.fit_transform(X_train_2[:, -7:])
X_test_2[:, -7:] = sc_2.transform(X_test_2[:, -7:])

#### Regression models

In [87]:
tree_2 = DecisionTreeRegressor()
tree_2.fit(X_train_2, y_train_2)
y_pred_2 = tree_2.predict(X_test_2)

In [88]:
r2_score(y_test_2, y_pred_2)

0.8479303410196957

In [94]:
forest_2 = RandomForestRegressor()
forest_2.fit(X_train_2, y_train_2)
y_pred_forest_2 = forest_2.predict(X_test_2)

In [95]:
r2_score(y_test_2, y_pred_forest_2)

0.8195603327663269

### Conclusion
- As expected, using year_1 data for training and year_2 data for testing is less efficient, because we are effectively loosing the information about the year. 
- However, the predictions are still quite high, meaning that overall the model is robust and we can use the entire dataset model

### PCA approach

In [96]:
from sklearn.decomposition import PCA

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68190 entries, 0 to 68189
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   project_name        68190 non-null  object        
 1   street              68190 non-null  object        
 2   postal_district     68190 non-null  int64         
 3   sale_type           68190 non-null  object        
 4   area_type           68190 non-null  object        
 5   property_type       68190 non-null  object        
 6   tenure_type         68190 non-null  object        
 7   mkt_segment         68190 non-null  object        
 8   sale_month          68190 non-null  int64         
 9   sale_year           68190 non-null  int64         
 10  sale_date           68190 non-null  datetime64[ns]
 11  tenure_start        68190 non-null  datetime64[ns]
 12  tenure_start_y_ago  68190 non-null  float64       
 13  tenure_years        68190 non-null  float64   

In [114]:
df_pca = df.drop(columns=['sale_date', 'tenure_start', 'tenure_years', 'lat', 'lon', 'price_total'])

In [115]:
df_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68190 entries, 0 to 68189
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_name        68190 non-null  object 
 1   street              68190 non-null  object 
 2   postal_district     68190 non-null  int64  
 3   sale_type           68190 non-null  object 
 4   area_type           68190 non-null  object 
 5   property_type       68190 non-null  object 
 6   tenure_type         68190 non-null  object 
 7   mkt_segment         68190 non-null  object 
 8   sale_month          68190 non-null  int64  
 9   sale_year           68190 non-null  int64  
 10  tenure_start_y_ago  68190 non-null  float64
 11  tenure_remained     68190 non-null  float64
 12  floor_level         68190 non-null  float64
 13  max_floor           68190 non-null  float64
 14  area_sqft           68190 non-null  float64
 15  lat_adj             68190 non-null  float64
 16  lon_

#### Column encoding

In [119]:
dataset_pca = df_pca.values
X_pca = dataset_pca[:, :-1]
y_pca = dataset_pca[:, -1]

In [120]:
columns_to_transform = [0, 1, 2, 3 , 4, 5, 6, 7, 8, 9]
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), columns_to_transform)], 
                       remainder='passthrough')
X_pca_ct = ct.fit_transform(X_pca).toarray()

#### Train_test split

In [122]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca_ct, y_pca, train_size=TRAIN_SIZE)

#### Feature scaling

In [123]:
#train
sc_pca = StandardScaler()
X_train_pca[:, -7:] = sc_pca.fit_transform(X_train_pca[:, -7:])

#test
X_test_pca[:, -7:] = sc_pca.transform(X_test_pca[:, -7:])

#### Running PCA

In [135]:
#train
pca = PCA(n_components=56)
X_train_pca_transformed = pca.fit_transform(X_train_pca)

In [136]:
#increasing the number of components until explained variance is 90%
pca.explained_variance_ratio_.cumsum()

array([0.21717893, 0.37852043, 0.45077998, 0.51573153, 0.57036805,
       0.60203081, 0.62764488, 0.64917147, 0.66856273, 0.68470654,
       0.69858351, 0.71069495, 0.72221395, 0.73198157, 0.74075578,
       0.74879195, 0.75646271, 0.76398011, 0.77104541, 0.7779921 ,
       0.78474553, 0.79128798, 0.79731208, 0.80316537, 0.8089406 ,
       0.81437913, 0.81968236, 0.82487921, 0.82986669, 0.83440376,
       0.83877598, 0.84280312, 0.84647744, 0.84997124, 0.85331465,
       0.85643278, 0.85939695, 0.86231037, 0.86505741, 0.8677039 ,
       0.87024725, 0.87272106, 0.87511667, 0.87748959, 0.87983045,
       0.88209423, 0.8843042 , 0.88640765, 0.88841833, 0.89035094,
       0.89224001, 0.89401998, 0.89576708, 0.89743615, 0.89904122,
       0.90059194])

In [137]:
#test
X_test_pca_transformed = pca.transform(X_test_pca)

#### Regression models on PCA

##### Linear regression

In [138]:
from sklearn.linear_model import LinearRegression

In [139]:
lr_pca = LinearRegression()

In [140]:
lr_pca.fit(X_train_pca_transformed, y_train_pca)

LinearRegression()

In [141]:
y_pred_lr_pca = lr_pca.predict(X_test_pca_transformed)

In [142]:
r2_score(y_test_pca, y_pred_lr_pca)

0.883844990666613

Comment:
- Although the original data contained significant non-linearity, Linear model can be used with PCA

##### Decision tree

In [144]:
tree_pca = DecisionTreeRegressor()
tree_pca.fit(X_train_pca_transformed, y_train_pca)
y_pred_tree_pca = tree_pca.predict(X_test_pca_transformed)
r2_score(y_test_pca, y_pred_tree_pca)

0.9200034909574065

##### Random forest

In [145]:
forest_pca = RandomForestRegressor()
forest_pca.fit(X_train_pca_transformed, y_train_pca)
y_pred_forest_pca = forest_pca.predict(X_test_pca_transformed)
r2_score(y_test_pca, y_pred_forest_pca)

0.9630431761807127

### Artificial Neural Network

##### Preparing the dataset

In [146]:
df_ann = df.drop(columns=['sale_date', 'tenure_start', 'tenure_years', 'lat', 'lon', 'price_total'])

In [147]:
df_ann.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68190 entries, 0 to 68189
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_name        68190 non-null  object 
 1   street              68190 non-null  object 
 2   postal_district     68190 non-null  int64  
 3   sale_type           68190 non-null  object 
 4   area_type           68190 non-null  object 
 5   property_type       68190 non-null  object 
 6   tenure_type         68190 non-null  object 
 7   mkt_segment         68190 non-null  object 
 8   sale_month          68190 non-null  int64  
 9   sale_year           68190 non-null  int64  
 10  tenure_start_y_ago  68190 non-null  float64
 11  tenure_remained     68190 non-null  float64
 12  floor_level         68190 non-null  float64
 13  max_floor           68190 non-null  float64
 14  area_sqft           68190 non-null  float64
 15  lat_adj             68190 non-null  float64
 16  lon_

In [183]:
dataset_ann = df_ann.values
X_ann = dataset_ann[:, :-1]
y_ann = dataset_ann[:, -1].astype('float32')

In [184]:
y_ann.dtype

dtype('float32')

##### Encoding categorical columns

In [185]:
columns_to_transform = [0, 1, 2, 3 , 4, 5, 6, 7, 8, 9]
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), columns_to_transform)], 
                       remainder='passthrough')
X_ann = ct.fit_transform(X_ann).toarray().astype('float32')

In [186]:
X_train_ann, X_test_ann, y_train_ann, y_test_ann = train_test_split(X_ann, y_ann, train_size=TRAIN_SIZE)

In [187]:
sc_ann = StandardScaler()
X_train_ann = sc_ann.fit_transform(X_train_ann)
X_test_ann = sc_ann.transform(X_test_ann)

In [189]:
y_train_ann.dtype

dtype('float32')

##### ANN

In [203]:
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError

In [204]:
ann = tf.keras.models.Sequential()

In [205]:
#Adding layers
ann.add(tf.keras.layers.Input(shape=(X_train_ann.shape[1])))
ann.add(tf.keras.layers.Dense(512, activation='relu'))  
ann.add(tf.keras.layers.Dense(256, activation='relu'))
ann.add(tf.keras.layers.Dense(128, activation='relu'))
ann.add(tf.keras.layers.Dense(1))

In [207]:
ann.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = [MeanSquaredError()])

In [208]:
#Training
ann.fit(X_train_ann, y_train_ann, batch_size = 32, epochs = 100)

Epoch 1/100
1492/1492 [==============================] - 7s 4ms/step - loss: 85973.3828 - mean_squared_error: 85973.3828
Epoch 2/100
1492/1492 [==============================] - 6s 4ms/step - loss: 13794.3896 - mean_squared_error: 13794.3896
Epoch 3/100
1492/1492 [==============================] - 6s 4ms/step - loss: 12858.9229 - mean_squared_error: 12858.9229
Epoch 4/100
1492/1492 [==============================] - 6s 4ms/step - loss: 11985.8135 - mean_squared_error: 11985.8135
Epoch 5/100
1492/1492 [==============================] - 6s 4ms/step - loss: 11283.1523 - mean_squared_error: 11283.1523
Epoch 6/100
1492/1492 [==============================] - 6s 4ms/step - loss: 10904.3584 - mean_squared_error: 10904.3584
Epoch 7/100
1492/1492 [==============================] - 6s 4ms/step - loss: 10209.6133 - mean_squared_error: 10209.6133
Epoch 8/100
1492/1492 [==============================] - 6s 4ms/step - loss: 9757.9785 - mean_squared_error: 9757.9785
Epoch 9/100
1492/1492 [===========

1492/1492 [==============================] - 7s 4ms/step - loss: 4142.0234 - mean_squared_error: 4142.0234: 0s - loss: 4081.
Epoch 69/100
1492/1492 [==============================] - 6s 4ms/step - loss: 4105.3345 - mean_squared_error: 4105.3345
Epoch 70/100
1492/1492 [==============================] - 7s 5ms/step - loss: 4105.0981 - mean_squared_error: 4105.0981
Epoch 71/100
1492/1492 [==============================] - 7s 5ms/step - loss: 4094.9333 - mean_squared_error: 4094.9333
Epoch 72/100
1492/1492 [==============================] - 9s 6ms/step - loss: 4015.9407 - mean_squared_error: 4015.9407
Epoch 73/100
1492/1492 [==============================] - 7s 4ms/step - loss: 4047.3352 - mean_squared_error: 4047.3352
Epoch 74/100
1492/1492 [==============================] - 6s 4ms/step - loss: 4011.9946 - mean_squared_error: 4011.9946
Epoch 75/100
1492/1492 [==============================] - 7s 4ms/step - loss: 4013.8071 - mean_squared_error: 4013.8071
Epoch 76/100
1492/1492 [===========

In [209]:
#Predicting

In [210]:
y_pred_ann = ann.predict(X_test_ann)

In [211]:
r2_score(y_test_ann, y_pred_ann)

0.9735127021005343